### Load preprocessed data

In [15]:
import numpy as np
fh = np.load('../data/dataset.npz')
# We have a bunch of feature columns and last column is the y-target
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']

test_x = fh['test_x'].astype(np.int64)
test_y = fh['test_y']

n_user = int(fh['n_user'])
n_item = int(fh['n_item'])
n_occu = int(fh['n_occu'])
n_rank = int(fh['n_ranks'])

### Define the MF Model

In [26]:
import torch
from torch import nn
import torch.nn.functional as F

def l2_regularize(array):
    loss = torch.sum(array ** 2.0)
    return loss


def total_variation(array):
    return torch.sum(torch.abs(array[:, :-1] - array[:, 1:]))


class MF(nn.Module):
    itr = 0
    
    def __init__(self, n_user, n_item, n_occu, n_rank, k=18, c_temp=1.0,
                 c_vector=1.0, c_bias=1.0, writer=None):
        super(MF, self).__init__()
        self.writer = writer
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_bias = c_bias
        self.c_vector = c_vector
        self.c_temp = c_temp
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)
        self.bias_user = nn.Embedding(n_user, 1)
        self.bias_item = nn.Embedding(n_item, 1)
        self.bias = nn.Parameter(torch.ones(1))
        self.occu = nn.Embedding(n_occu, k)
        self.temp = nn.Embedding(n_rank, k)
        
        # **NEW: temporal vectors
        self.user_temp = nn.Embedding(n_user, k)
        self.temp = nn.Embedding(n_rank, k)

    
    def __call__(self, train_x):
        user_id = train_x[:, 0]
        item_id = train_x[:, 1]
        occu_id = train_x[:, 3]
        vector_user = self.user(user_id)
        vector_item = self.item(item_id)
        vector_occu = self.occu(occu_id)
        bias_user = self.bias_user(user_id).squeeze()
        bias_item = self.bias_item(item_id).squeeze()
        biases = (self.bias + bias_user + bias_item)
        ui_interaction = torch.sum(vector_user * vector_item, dim=1)
        oi_interaction = torch.sum(vector_user * vector_occu, dim=1)
        
        # **NEW: user-time interaction
        rank = train_x[:, 2]
        vector_user_temp = self.user_temp(user_id)
        vector_temp = self.temp(rank)
        ut_interaction = torch.sum(vector_user_temp * vector_temp, dim=1)
        
        prediction = ui_interaction + oi_interaction + ut_interaction + biases
        return prediction
    
    def loss(self, prediction, target):
        loss_mse = F.mse_loss(prediction.squeeze(), target.squeeze())
        prior_bias_user =  l2_regularize(self.bias_user.weight) * self.c_bias
        prior_bias_item = l2_regularize(self.bias_item.weight) * self.c_bias
        prior_user =  l2_regularize(self.user.weight) * self.c_vector
        prior_item = l2_regularize(self.item.weight) * self.c_vector
        prior_occu = l2_regularize(self.item.weight) * self.c_vector
        
        # New: total variation regularization
        prior_ut = l2_regularize(self.user_temp.weight) * self.c_vector
        prior_tv = total_variation(self.temp.weight) * self.c_temp
        
        total = (loss_mse + prior_user + prior_item +
                 prior_bias_item + prior_bias_user +  prior_occu + prior_tv)
        for name, var in locals().items():
            if type(var) is torch.Tensor and var.nelement() == 1 and self.writer is not None:
                self.writer.add_scalar(name, var, self.itr)
        return total

### Train model

In [27]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss
from tensorboardX import SummaryWriter
from ignite.metrics import MeanSquaredError

from loader import Loader

#### Hyperparameters

In [30]:
n_epochs = 10
k = 10
c_bias = 1e-6
c_vector = 1e-6
c_temp = 1e-6

In [31]:
writer = SummaryWriter()
model = MF(n_user, n_item, n_occu, n_rank, k=k, c_bias=c_bias, 
           c_vector=c_vector, c_temp=c_temp, writer=writer)
optimizer = torch.optim.Adam(model.parameters())
trainer = create_supervised_trainer(model, optimizer, model.loss)
metrics = {'accuracy': MeanSquaredError()}
evaluat = create_supervised_evaluator(model, metrics=metrics)
train_loader = Loader(train_x, train_y, batchsize=1024)
test_loader = Loader(test_x, test_y, batchsize=1024)


def log_training_loss(engine, log_interval=400):
    epoch = engine.state.epoch
    itr = engine.state.iteration
    fmt = "Epoch[{}] Iteration[{}/{}] Loss: {:.2f}"
    msg = fmt.format(epoch, itr, len(train_loader), engine.state.output)
    model.itr = itr
    if itr % log_interval == 0:
        print(msg)

trainer.add_event_handler(event_name=Events.ITERATION_COMPLETED, handler=log_training_loss)

def log_validation_results(engine):
    evaluat.run(test_loader)
    metrics = evaluat.state.metrics
    avg_accuracy = metrics['accuracy']
    print("Epoch[{}] Validation MSE: {:.2f} "
          .format(engine.state.epoch, avg_accuracy))
    writer.add_scalar("validation/avg_accuracy", avg_accuracy, engine.state.epoch)

trainer.add_event_handler(event_name=Events.EPOCH_COMPLETED, handler=log_validation_results)


model

MF(
  (user): Embedding(6041, 10)
  (item): Embedding(3953, 10)
  (bias_user): Embedding(6041, 1)
  (bias_item): Embedding(3953, 1)
  (occu): Embedding(21, 10)
  (temp): Embedding(2315, 10)
  (user_temp): Embedding(6041, 10)
)

#### Run model

In [ ]:
trainer.run(train_loader, max_epochs=50)